## Import Libraries

In [1]:
import cv2
import PIL
import glob
import time
import scipy
import random
import pathlib
import warnings
import os,shutil
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
from scipy import ndimage
from tensorflow import keras
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
from tensorflow.keras import layers
from skimage import io, color, feature
from keras.utils import to_categorical
from sklearn.metrics import roc_curve, auc
from keras.metrics import Precision, Recall
from tensorflow.keras.regularizers import l2
from sklearn.metrics import confusion_matrix
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping
from skimage.feature import graycomatrix, graycoprops
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16, ResNet50
from sklearn.utils.class_weight import compute_class_weight
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras.optimizers.schedules import ExponentialDecay
from tensorflow.keras.utils import array_to_img, img_to_array, load_img
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout, LeakyReLU, BatchNormalization, Activation
warnings.filterwarnings("ignore")

ImportError: Traceback (most recent call last):
  File "C:\Users\parth\AppData\Local\Programs\Python\Python312\Lib\site-packages\tensorflow\python\pywrap_tensorflow.py", line 73, in <module>
    from tensorflow.python._pywrap_tensorflow_internal import *
ImportError: DLL load failed while importing _pywrap_tensorflow_internal: A dynamic link library (DLL) initialization routine failed.


Failed to load the native TensorFlow runtime.
See https://www.tensorflow.org/install/errors for some common causes and solutions.
If you need help, create an issue at https://github.com/tensorflow/tensorflow/issues and include the entire stack trace above this error message.

## Load Dataset

In [2]:
train_data_dir = pathlib.Path("dataset/Train")
test_data_dir = pathlib.Path("dataset/Test")

# Counting Test and Train sets

In [3]:

image_count_train = len(list(train_data_dir.glob('*/*.jpg')))
print("Train images:", image_count_train)
image_count_test = len(list(test_data_dir.glob('*/*.jpg')))
print("Test images:", image_count_test)

Train images: 2239
Test images: 118


# Counting images from each class

In [4]:

class_names = sorted([dir.name for dir in train_data_dir.glob('*')])

class_names_list = []
train_counts = []
test_counts = []
for class_name in class_names:
    train_class_dir = train_data_dir / class_name
    test_class_dir = test_data_dir / class_name

    train_count = len(list(train_class_dir.glob('*.jpg')))
    test_count = len(list(test_class_dir.glob('*.jpg')))

    class_names_list.append(class_name)
    train_counts.append(train_count)
    test_counts.append(test_count)

# Create a Pandas DataFrame
data = {
    "Class Name": class_names_list,
    "Train Images": train_counts,
    "Test Images": test_counts
}
df = pd.DataFrame(data)
df.reset_index(inplace=True, drop=True)

print(df)

                   Class Name  Train Images  Test Images
0           actinic keratosis           114           16
1        basal cell carcinoma           376           16
2              dermatofibroma            95           16
3                    melanoma           438           16
4                       nevus           357           16
5  pigmented benign keratosis           462           16
6        seborrheic keratosis            77            3
7     squamous cell carcinoma           181           16
8             vascular lesion           139            3


# Visualization of Train and Test sets

In [5]:

image_count_train = len(list(train_data_dir.glob('*/*.jpg')))
image_count_test = len(list(test_data_dir.glob('*/*.jpg')))

categories = ['Train', 'Test']
counts = [image_count_train, image_count_test]

plt.figure(figsize=(8, 6))
plt.bar(categories, counts, color=['blue', 'red'])
plt.xlabel('Dataset')
plt.ylabel('Image Count')
plt.title('Train vs. Test Image Counts')
plt.show()

NameError: name 'plt' is not defined

In [ ]:
# Creating a new column 'Cancerous' to categorize conditions as cancerous or non-cancerous
df['Cancerous'] = df['Class Name'].apply(lambda x: 'Cancerous' if x in ['melanoma', 'basal cell carcinoma', 'squamous cell carcinoma'] else 'Non-Cancerous')

# Counting the number of cancerous and non-cancerous conditions in the training set
train_cancerous_count = df[df['Cancerous'] == 'Cancerous']['Train Images'].sum()
train_non_cancerous_count = df[df['Cancerous'] == 'Non-Cancerous']['Train Images'].sum()

# Counting the number of cancerous and non-cancerous conditions in the test set
test_cancerous_count = df[df['Cancerous'] == 'Cancerous']['Test Images'].sum()
test_non_cancerous_count = df[df['Cancerous'] == 'Non-Cancerous']['Test Images'].sum()

# Calculating the total counts of cancerous and non-cancerous conditions
total_cancerous_count = train_cancerous_count + test_cancerous_count
total_non_cancerous_count = train_non_cancerous_count + test_non_cancerous_count

print("Cancerous Skin Conditions (Train):", train_cancerous_count)
print("Non-Cancerous Skin Conditions (Train):", train_non_cancerous_count)
print("Cancerous Skin Conditions (Test):", test_cancerous_count)
print("Non-Cancerous Skin Conditions (Test):", test_non_cancerous_count)
print("Total Cancerous Skin Conditions:", total_cancerous_count)
print("Total Non-Cancerous Skin Conditions:", total_non_cancerous_count)

In [ ]:
categories = ['Train (Cancerous)', 'Train (Non-Cancerous)', 'Test (Cancerous)', 'Test (Non-Cancerous)']
counts = [train_cancerous_count, train_non_cancerous_count, test_cancerous_count, test_non_cancerous_count]

# Calculating total counts
total_cancerous_count = train_cancerous_count + test_cancerous_count
total_non_cancerous_count = train_non_cancerous_count + test_non_cancerous_count
total_counts = [total_cancerous_count, total_non_cancerous_count, total_cancerous_count, total_non_cancerous_count]

# Stacked horizontal bar chart
plt.figure(figsize=(8, 4))
bottom = [0, 0, 0, 0]
colors = ['red', 'blue', 'red', 'blue']
bar_width = 0.5

for i in range(len(categories)):
    plt.barh(categories[i], counts[i], height=bar_width, left=bottom[i], color=colors[i], label=categories[i])
    bottom[i] += counts[i]

plt.title('Skin Condition Counts (Train and Test)')
plt.ylabel('Category')
plt.xlabel('Count')
plt.legend()
plt.show()

In [ ]:
# Random dispay of cancerous images
plt.figure(figsize=(10, 10))
cancerous_indices = [i for i, class_name in enumerate(class_names) if class_name in ['melanoma', 'basal cell carcinoma', 'squamous cell carcinoma']]

for i, idx in enumerate(cancerous_indices):
    plt.subplot(3, 3, i + 4)
    class_dir = random.choice(list(train_data_dir.glob(class_names[idx] + '/*.jpg')))
    image = plt.imread(str(class_dir))
    plt.title(class_names[idx])
    plt.imshow(image)
    plt.axis("off")
plt.show()

In [ ]:
# Random display of non-cancerous images
plt.figure(figsize=(10, 10))
non_cancerous_indices = [i for i in range(len(class_names)) if i not in cancerous_indices]

for i, idx in enumerate(non_cancerous_indices):
    plt.subplot(3, 3, i + 4)
    class_dir = random.choice(list(train_data_dir.glob(class_names[idx] + '/*.jpg')))
    image = plt.imread(str(class_dir))
    plt.title(class_names[idx])
    plt.imshow(image)
    plt.axis("off")

plt.show()

In [ ]:
plt.figure(figsize=(10, 10))
for i in range(9):
    plt.subplot(3, 3, i + 1)
    class_dir = random.choice(list(train_data_dir.glob(class_names[i] + '/*.jpg')))
    image = plt.imread(str(class_dir))
    plt.title(class_names[i])
    plt.imshow(image)
    plt.axis("off")
plt.show()

In [ ]:
columns_to_plot = ["Train Images", "Test Images" ]

def plot_bar_chart(column):
    plt.figure(figsize=(10, 5))
    sns.barplot(data=df, y='Class Name', x=column, orient ='h')
    plt.title(f"Count of {column} per Class")
    plt.xlabel("Count")
    plt.ylabel("Class Name")
    plt.xticks(rotation=45, ha="right")
    plt.tight_layout()
    plt.show()

for column in columns_to_plot:
    plot_bar_chart(column)


In [ ]:
def create_histogram(data, title, color):
    plt.hist(data, bins=20, color=color, alpha=0.7)
    plt.xlabel(title)
    plt.ylabel('Frequency')
    plt.title(f'Distribution of {title}s')

# Defining subdirectories
subdirectories = [train_data_dir, test_data_dir]

image_widths = []
image_heights = []

for subset_dir in subdirectories:
    subset_name = os.path.basename(subset_dir)
    classes = os.listdir(subset_dir)

    for class_name in classes:
        class_dir = os.path.join(subset_dir, class_name)
        sample_images = os.listdir(class_dir)

        for image_name in sample_images:
            image_path = os.path.join(class_dir, image_name)
            img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
            height, width = img.shape[:2]
            image_widths.append(width)
            image_heights.append(height)

# Creating histograms for image widths and heights
plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
create_histogram(image_widths, 'Image Width', 'blue')

plt.subplot(1, 2, 2)
create_histogram(image_heights, 'Image Height', 'red')
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(11, 5))
plt.subplot(1, 2, 1)
plt.hexbin(x=image_widths, y=image_heights, gridsize=20, cmap='Reds', vmin=0, vmax=100)
plt.xlabel('Image Width')
plt.ylabel('Image Height')
plt.title('Hexbin Plot of Image Dimensions (Width vs. Height)')
plt.tight_layout()
plt.show()

In [ ]:
# Defining the classes in the data directory
classes = os.listdir(train_data_dir)

feature_vectors = []
class_labels = []

def extract_features(image):
    resized_image = cv2.resize(image, (64, 64))
    feature_vector = resized_image.flatten()
    return feature_vector

for i, class_name in enumerate(classes):
    class_dir = os.path.join(train_data_dir, class_name)
    sample_images = os.listdir(class_dir)

    for image_name in sample_images:
        image_path = os.path.join(class_dir, image_name)
        img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

        feature_vector = extract_features(img)

        feature_vectors.append(feature_vector)
        class_labels.append(i)

# Standardizing the features vectors (mean=0, std=1)
scaler = StandardScaler()
feature_vectors = scaler.fit_transform(feature_vectors)

# Applying t-SNE to reduce dimensionality to 2D
tsne = TSNE(n_components=2, random_state=42)
embedded_features = tsne.fit_transform(feature_vectors)

# Creating a scatter plot to visualize class separation
plt.figure(figsize=(8, 6))
plt.scatter(embedded_features[:, 0], embedded_features[:, 1], c=class_labels, cmap='viridis')
plt.title("t-SNE Visualization of Class Separation")
plt.colorbar()
plt.show()


In [ ]:
# List of class folders containing images
class_folders = os.listdir(train_data_dir)

# GLCM properties to calculate
properties = ['contrast', 'dissimilarity', 'homogeneity', 'energy', 'correlation']

# Creating an empty dictionary to store histograms for each property
property_histograms = {prop: [] for prop in properties}

# Calculating GLCM Features for Each Image in Each Class:
for folder in class_folders:
    class_path = os.path.join(train_data_dir, folder)
    image_files = os.listdir(class_path)

    for image_file in image_files:
        image_path = os.path.join(class_path, image_file)
        image = io.imread(image_path)
        gray_image = color.rgb2gray(image)
        gray_image = (gray_image * 255).astype(np.uint8)
        glcm = graycomatrix(gray_image, [1], [0], symmetric=True, normed=True)
        features = [feature.graycoprops(glcm, prop)[0, 0] for prop in properties]

        for prop, feature_value in zip(properties, features):
            property_histograms[prop].append((folder, feature_value))

# Creating subplots for each property
num_properties = len(properties)
fig, axes = plt.subplots(1, num_properties, figsize=(20, 5))
for i, prop in enumerate(properties):
    ax = axes[i]
    ax.set_xlabel(prop.capitalize())
    ax.set_ylabel('Frequency')
    ax.set_title(f'{prop.capitalize()} Histograms')

    # Plotting histograms for each class
    for folder in class_folders:
        subset = [value for label, value in property_histograms[prop] if label == folder]
        ax.hist(subset, bins=20, alpha=0.5, label=folder)

    ax.legend(title='Class')


plt.tight_layout()

plt.show()

In [ ]:
# Number of random images to display
num_images_to_display = 4

# Create a function to plot color channels
def plot_color_channels(image, class_name):
    # Convert BGR to RGB
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Split the image into color channels
    blue_channel, green_channel, red_channel = cv2.split(image)

    # Create subplots for displaying color channels
    fig, axes = plt.subplots(1, 4, figsize=(16, 4))
    axes[0].imshow(image)
    axes[0].set_title(class_name)
    axes[0].axis("off")
    for i, (channel, cmap_name) in enumerate(zip([red_channel, green_channel, blue_channel], ['Reds', 'Greens', 'Blues'])):
        axes[i+1].imshow(channel, cmap=cmap_name)
        axes[i+1].set_title(f"{class_name} - {cmap_name.split('_')[0].capitalize()} Channel")
        axes[i+1].axis("off")

# Loop to display random images with color channels for three different classes
for i in range(3):  # Iterate over three different class indices
    # Get a list of image files in the class directory
    class_dir = list(train_data_dir.glob(class_names[i] + '/*.jpg'))

    if not class_dir:
        # Handle the case where there are no images in the directory
        print(f"No images found in the {class_names[i]} directory.")
    else:
        # Choose a random image from the list
        random_image = random.choice(class_dir)
        image = cv2.imread(str(random_image))

        # Call the function to plot color channels
        plot_color_channels(image, class_names[i])

plt.tight_layout()
plt.show()

In [ ]:
base_directory = 'dataset/Train'
# Creating a pattern to match image files (e.g., all files ending with '.jpg')
image_pattern = '*.jpg'

# list of class subdirectories
class_directories = glob.glob(base_directory + '*/')


for class_dir in class_directories:
    class_name = os.path.basename(os.path.normpath(class_dir))
    image_paths = glob.glob(class_dir + image_pattern)

    for image_path in image_paths:
        # Loading an image using OpenCV
        image = cv2.imread(image_path)

       # Converting the image to grayscale
gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Computing histogram of grayscale pixel intensities
hist = cv2.calcHist([gray_image], [0], None, [256], [0, 256])

plt.figure(figsize=(10, 6))
plt.hist(gray_image.ravel(), bins=256, range=(0, 256), density=True, color='b', alpha=0.6)
plt.title('Grayscale Pixel Intensity Distribution')
plt.xlabel('Pixel Value')
plt.ylabel('Normalized Frequency')
plt.show()


In [ ]:
# Defining the destination directories for the training and validation sets
training_data_dir = pathlib.Path("dataset/split_data/Training_set")
validation_data_dir = pathlib.Path("dataset/split_dat/Validation_set")

# Creating the destination directories
training_data_dir.mkdir(parents=True, exist_ok=True)
validation_data_dir.mkdir(parents=True, exist_ok=True)

class_dirs = [dir.name for dir in test_data_dir.glob('*')]

# Splitting the data into training and validation sets while maintaining the original directory structure
for class_dir in class_dirs:
    class_images = list((train_data_dir / class_dir).glob('*.jpg'))
    train_images, val_images = train_test_split(class_images, test_size=0.4, random_state=123)
    (training_data_dir / class_dir).mkdir(parents=True, exist_ok=True)
    (validation_data_dir / class_dir).mkdir(parents=True, exist_ok=True)

    # Copying the images to their respective directories
    for train_image in train_images:
        shutil.copy(train_image, training_data_dir / class_dir / train_image.name)

    for val_image in val_images:
        shutil.copy(val_image, validation_data_dir / class_dir / val_image.name)

print("Data splitting complete.")

In [ ]:
#Total Count for created Training_set, Validation_set and Original Test
image_count_training = len(list(training_data_dir.glob('*/*.jpg')))
print("Training images:", image_count_training)
image_count_validation = len(list(validation_data_dir.glob('*/*.jpg')))
print("Validation images:", image_count_validation)
image_count_test = len(list(test_data_dir.glob('*/*.jpg')))
print("Test images:", image_count_test)

In [ ]:
# transforming data in the directory split/train (1574 images)
batch_size =32
class_mode = 'categorical'
train_generator = ImageDataGenerator(rescale=1./255).flow_from_directory(
        training_data_dir,
        target_size=(64, 64), batch_size = batch_size, class_mode = class_mode)

# transforming  data in the directory split/validation (677 images)
val_generator = ImageDataGenerator(rescale=1./255).flow_from_directory(
        validation_data_dir,
        target_size=(64, 64), batch_size = batch_size, class_mode = class_mode)

# tranforming the data in the directory test (118 images)
test_generator = ImageDataGenerator(rescale=1./255).flow_from_directory(
        test_data_dir,
        target_size=(64, 64), batch_size=batch_size, class_mode = class_mode)

In [ ]:
# class labels and counts for the training set
class_labels = train_generator.class_indices
class_counts = train_generator.classes

# Calculating the number of samples in each class
unique_classes, class_counts = np.unique(class_counts, return_counts=True)

# Mapping class labels to class names
class_names = {v: k for k, v in class_labels.items()}

plt.figure(figsize=(10, 5))
plt.bar(class_names.values(), class_counts)
plt.title('Class Distribution in Training Set')
plt.xlabel('Class')
plt.ylabel('Number of Samples')
plt.xticks(rotation=45, ha="right")  # Rotate x-axis labels for readability
plt.tight_layout()
plt.show()

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras.metrics import Precision, Recall

def create_and_train_model(model_number, train_generator, val_generator):
    # Creating the model layers
    np.random.seed(123)
    model = Sequential()
    model.add(layers.Conv2D(16, (3, 3), activation='relu', padding='same',
                            input_shape=(64, 64, 3)))
    model.add(layers.MaxPooling2D((2, 2)))

    model.add(layers.Conv2D(32, (4, 4), activation='relu', padding='same'))
    model.add(layers.MaxPooling2D((2, 2)))

    model.add(layers.Conv2D(64, (3, 3), activation='relu', padding='same'))
    model.add(layers.MaxPooling2D((2, 2)))

    model.add(layers.Flatten())
    model.add(layers.Dense(128, activation='relu'))
    model.add(layers.Dense(9, activation='softmax'))

    model.compile(loss='categorical_crossentropy',
                  optimizer="adam",
                  metrics=['accuracy', Precision(), Recall()])

    # Training the model without early stopping
    history = model.fit(train_generator,
                        epochs=100,
                        validation_data=val_generator)

    return model, history


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers

model1, history1 = create_and_train_model(1, train_generator, val_generator)

In [ ]:
model1.save("CNN_model.h5")

In [ ]:
from tensorflow.keras.models import load_model

# Load the saved baseline model
baseline_model = load_model("CNN_model.h5")

# Visualize the loaded baseline model
from tensorflow.keras.utils import plot_model

# Plot the baseline model architecture
plot_model(baseline_model, to_file='baseline_model.png', show_shapes=True)

In [ ]:
# Let us have a close look at precision of the above model
precision = model1.evaluate(train_generator, verbose=1)[2]
precision_v = model1.evaluate(val_generator, verbose=1)[2]
print("Precision: ", precision)
print("Validation Precision: ", precision_v)

In [ ]:
import matplotlib.pyplot as plt

# Print available keys in history to identify actual metric names
print("Available keys in history:", history1.history.keys())

# Accessing loss and accuracy (fixed names)
training_loss = history1.history['loss']
validation_loss = history1.history['val_loss']
training_accuracy = history1.history['accuracy']
validation_accuracy = history1.history['val_accuracy']

# Detect correct precision and recall keys
precision_key = [k for k in history1.history.keys() if 'precision' in k and not 'val' in k][0]
val_precision_key = [k for k in history1.history.keys() if 'val_precision' in k][0]
recall_key = [k for k in history1.history.keys() if 'recall' in k and not 'val' in k][0]
val_recall_key = [k for k in history1.history.keys() if 'val_recall' in k][0]

# Accessing precision and recall values
training_precision = history1.history[precision_key]
validation_precision = history1.history[val_precision_key]
training_recall = history1.history[recall_key]
validation_recall = history1.history[val_recall_key]

# Number of epochs
epochs = range(1, len(training_loss) + 1)

# Create a plot
plt.figure(figsize=(12, 6))

# Plot Loss
plt.subplot(1, 2, 1)
plt.plot(epochs, training_loss, 'b-', label='Train Loss')
plt.plot(epochs, validation_loss, 'r-', label='Val Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

# Plot Accuracy, Precision, Recall
plt.subplot(1, 2, 2)
plt.plot(epochs, training_accuracy, 'b-', label='Train Accuracy')
plt.plot(epochs, validation_accuracy, 'r-', label='Val Accuracy')
plt.plot(epochs, training_precision, 'g-', label='Train Precision')
plt.plot(epochs, validation_precision, 'c-', label='Val Precision')
plt.plot(epochs, training_recall, 'm-', label='Train Recall')
plt.plot(epochs, validation_recall, 'y-', label='Val Recall')
plt.title('Training and Validation Metrics')
plt.xlabel('Epochs')
plt.ylabel('Score')
plt.legend()

plt.tight_layout()
plt.show()


## VGG16 

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    'dataset/train',                # Replace with your actual path
    target_size=(224, 224),         # IMPORTANT: Must match VGG16 input
    batch_size=32,
    class_mode='categorical'
)

val_generator = val_datagen.flow_from_directory(
    'dataset/test',                  # Replace with your actual path
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)


In [ ]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GlobalAveragePooling2D, Dropout, Dense
from tensorflow.keras.metrics import Precision, Recall
from tensorflow.keras.optimizers import Adam
import numpy as np
import tensorflow as tf

# Set random seed
np.random.seed(123)

# Load base VGG16
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False  # Freeze base model

# Create model
model4 = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dropout(0.5),
    Dense(256, activation='relu'),
    Dense(128, activation='relu'),
    Dense(64, activation='relu'),
    Dense(9, activation='softmax')  # 9 class labels
])

# Compile
model4.compile(
    optimizer=Adam(learning_rate=0.0001),
    loss='categorical_crossentropy',
    metrics=['accuracy', Precision(), Recall()]
)

# Print summary
model4.summary()


In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=10,
    restore_best_weights=True
)

history4 = model4.fit(
    train_generator,
    validation_data=val_generator,
    epochs=100,
    callbacks=[early_stopping]
)


In [ ]:
import tkinter as tk
from tkinter import filedialog, Label, Button
from PIL import Image, ImageTk
import numpy as np
from tensorflow.keras.models import load_model

# Load your trained model
model = load_model('CNN_model.h5')  # Update this path if different

# Updated class labels
class_labels = [
    "actinic keratosis",
    "basal cell carcinoma",
    "dermatofibroma",
    "melanoma",
    "nevus",
    "pigmented benign keratosis",
    "seborrheic keratosis",
    "squamous cell carcinoma",
    "vascular lesion"
]

# Image preprocessing
def preprocess_image(image_path):
    img = Image.open(image_path).resize((64, 64))
    img_array = np.array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)
    return img_array

# Predict and display
def predict_image():
    file_path = filedialog.askopenfilename()
    if not file_path:
        return

    img_array = preprocess_image(file_path)
    prediction = model.predict(img_array)
    predicted_class = class_labels[np.argmax(prediction)]

    # Show image and result
    img = Image.open(file_path).resize((200, 200))
    img_tk = ImageTk.PhotoImage(img)
    image_label.config(image=img_tk)
    image_label.image = img_tk
    result_label.config(text=f"Prediction: {predicted_class}")

# GUI setup
window = tk.Tk()
window.title("Skin Disease Classifier - Manual Test")
window.geometry("420x450")

Button(window, text="Choose Image and Predict", command=predict_image, font=("Arial", 12), bg="#add8e6").pack(pady=20)
image_label = Label(window)
image_label.pack(pady=10)
result_label = Label(window, text="", font=("Arial", 13, "bold"))
result_label.pack(pady=10)

window.mainloop()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from tensorflow.keras.models import load_model
from tkinter import Tk, filedialog
import os

# Hide the Tkinter root window
Tk().withdraw()

# Load the trained model
model = load_model('CNN_model.h5')

# Class labels
class_labels = [
    "actinic keratosis",
    "basal cell carcinoma",
    "dermatofibroma",
    "melanoma",
    "nevus",
    "pigmented benign keratosis",
    "seborrheic keratosis",
    "squamous cell carcinoma",
    "vascular lesion"
]

# File picker dialog
file_path = filedialog.askopenfilename(title="Select a skin image", filetypes=[("Image files", "*.jpg *.jpeg *.png")])

if file_path:
    # Load and preprocess image
    img = Image.open(file_path).resize((64, 64))
    img_array = np.array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)

    # Predict
    prediction = model.predict(img_array)
    predicted_class = class_labels[np.argmax(prediction)]
    confidence = np.max(prediction) * 100

    # Display result
    plt.imshow(Image.open(file_path))
    plt.axis('off')
    plt.title(f"Prediction: {predicted_class} ({confidence:.2f}%)")
    plt.show()
else:
    print("No image selected.")


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from tensorflow.keras.models import load_model
from tkinter import Tk, filedialog
import os

# Hide the Tkinter root window
Tk().withdraw()

# Load the trained model
model = load_model('CNN_model.h5')

# Class labels
class_labels = [
    "actinic keratosis",
    "basal cell carcinoma",
    "dermatofibroma",
    "melanoma",
    "nevus",
    "pigmented benign keratosis",
    "seborrheic keratosis",
    "squamous cell carcinoma",
    "vascular lesion"
]

# File picker dialog
file_path = filedialog.askopenfilename(title="Select a skin image", filetypes=[("Image files", "*.jpg *.jpeg *.png")])

if file_path:
    # Load and preprocess image
    img = Image.open(file_path).resize((64, 64))
    img_array = np.array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)

    # Predict
    prediction = model.predict(img_array)
    predicted_class = class_labels[np.argmax(prediction)]
    confidence = np.max(prediction) * 100

    # Display result
    plt.imshow(Image.open(file_path))
    plt.axis('off')
    plt.title(f"Prediction: {predicted_class} ({confidence:.2f}%)")
    plt.show()
else:
    print("No image selected.")


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from tensorflow.keras.models import load_model
from tkinter import Tk, filedialog
import os

# Hide the Tkinter root window
Tk().withdraw()

# Load the trained model
model = load_model('CNN_model.h5')

# Class labels
class_labels = [
    "actinic keratosis",
    "basal cell carcinoma",
    "dermatofibroma",
    "melanoma",
    "nevus",
    "pigmented benign keratosis",
    "seborrheic keratosis",
    "squamous cell carcinoma",
    "vascular lesion"
]

# File picker dialog
file_path = filedialog.askopenfilename(title="Select a skin image", filetypes=[("Image files", "*.jpg *.jpeg *.png")])

if file_path:
    # Load and preprocess image
    img = Image.open(file_path).resize((64, 64))
    img_array = np.array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)

    # Predict
    prediction = model.predict(img_array)
    predicted_class = class_labels[np.argmax(prediction)]
    confidence = np.max(prediction) * 100

    # Display result
    plt.imshow(Image.open(file_path))
    plt.axis('off')
    plt.title(f"Prediction: {predicted_class} ({confidence:.2f}%)")
    plt.show()
else:
    print("No image selected.")


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from tensorflow.keras.models import load_model
from tkinter import Tk, filedialog
import os

# Hide the Tkinter root window
Tk().withdraw()

# Load the trained model
model = load_model('CNN_model.h5')

# Class labels
class_labels = [
    "actinic keratosis",
    "basal cell carcinoma",
    "dermatofibroma",
    "melanoma",
    "nevus",
    "pigmented benign keratosis",
    "seborrheic keratosis",
    "squamous cell carcinoma",
    "vascular lesion"
]

# File picker dialog
file_path = filedialog.askopenfilename(title="Select a skin image", filetypes=[("Image files", "*.jpg *.jpeg *.png")])

if file_path:
    # Load and preprocess image
    img = Image.open(file_path).resize((64, 64))
    img_array = np.array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)

    # Predict
    prediction = model.predict(img_array)
    predicted_class = class_labels[np.argmax(prediction)]
    confidence = np.max(prediction) * 100

    # Display result
    plt.imshow(Image.open(file_path))
    plt.axis('off')
    plt.title(f"Prediction: {predicted_class} ({confidence:.2f}%)")
    plt.show()
else:
    print("No image selected.")


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from tensorflow.keras.models import load_model
from tkinter import Tk, filedialog
import os

# Hide the Tkinter root window
Tk().withdraw()

# Load the trained model
model = load_model('CNN_model.h5')

# Class labels
class_labels = [
    "actinic keratosis",
    "basal cell carcinoma",
    "dermatofibroma",
    "melanoma",
    "nevus",
    "pigmented benign keratosis",
    "seborrheic keratosis",
    "squamous cell carcinoma",
    "vascular lesion"
]

# File picker dialog
file_path = filedialog.askopenfilename(title="Select a skin image", filetypes=[("Image files", "*.jpg *.jpeg *.png")])

if file_path:
    # Load and preprocess image
    img = Image.open(file_path).resize((64, 64))
    img_array = np.array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)

    # Predict
    prediction = model.predict(img_array)
    predicted_class = class_labels[np.argmax(prediction)]
    confidence = np.max(prediction) * 100

    # Display result
    plt.imshow(Image.open(file_path))
    plt.axis('off')
    plt.title(f"Prediction: {predicted_class} ({confidence:.2f}%)")
    plt.show()
else:
    print("No image selected.")


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from tensorflow.keras.models import load_model
from tkinter import Tk, filedialog
import os

# Hide the Tkinter root window
Tk().withdraw()

# Load the trained model
model = load_model('CNN_model.h5')

# Class labels
class_labels = [
    "actinic keratosis",
    "basal cell carcinoma",
    "dermatofibroma",
    "melanoma",
    "nevus",
    "pigmented benign keratosis",
    "seborrheic keratosis",
    "squamous cell carcinoma",
    "vascular lesion"
]

# File picker dialog
file_path = filedialog.askopenfilename(title="Select a skin image", filetypes=[("Image files", "*.jpg *.jpeg *.png")])

if file_path:
    # Load and preprocess image
    img = Image.open(file_path).resize((64, 64))
    img_array = np.array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)

    # Predict
    prediction = model.predict(img_array)
    predicted_class = class_labels[np.argmax(prediction)]
    confidence = np.max(prediction) * 100

    # Display result
    plt.imshow(Image.open(file_path))
    plt.axis('off')
    plt.title(f"Prediction: {predicted_class} ({confidence:.2f}%)")
    plt.show()
else:
    print("No image selected.")


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from tensorflow.keras.models import load_model
from tkinter import Tk, filedialog
import os

# Hide the Tkinter root window
Tk().withdraw()

# Load the trained model
model = load_model('CNN_model.h5')

# Class labels
class_labels = [
    "actinic keratosis",
    "basal cell carcinoma",
    "dermatofibroma",
    "melanoma",
    "nevus",
    "pigmented benign keratosis",
    "seborrheic keratosis",
    "squamous cell carcinoma",
    "vascular lesion"
]

# File picker dialog
file_path = filedialog.askopenfilename(title="Select a skin image", filetypes=[("Image files", "*.jpg *.jpeg *.png")])

if file_path:
    # Load and preprocess image
    img = Image.open(file_path).resize((64, 64))
    img_array = np.array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)

    # Predict
    prediction = model.predict(img_array)
    predicted_class = class_labels[np.argmax(prediction)]
    confidence = np.max(prediction) * 100

    # Display result
    plt.imshow(Image.open(file_path))
    plt.axis('off')
    plt.title(f"Prediction: {predicted_class} ({confidence:.2f}%)")
    plt.show()
else:
    print("No image selected.")


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from tensorflow.keras.models import load_model
from tkinter import Tk, filedialog
import os

# Hide the Tkinter root window
Tk().withdraw()

# Load the trained model
model = load_model('CNN_model.h5')

# Class labels
class_labels = [
    "actinic keratosis",
    "basal cell carcinoma",
    "dermatofibroma",
    "melanoma",
    "nevus",
    "pigmented benign keratosis",
    "seborrheic keratosis",
    "squamous cell carcinoma",
    "vascular lesion"
]

# File picker dialog
file_path = filedialog.askopenfilename(title="Select a skin image", filetypes=[("Image files", "*.jpg *.jpeg *.png")])

if file_path:
    # Load and preprocess image
    img = Image.open(file_path).resize((64, 64))
    img_array = np.array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)

    # Predict
    prediction = model.predict(img_array)
    predicted_class = class_labels[np.argmax(prediction)]
    confidence = np.max(prediction) * 100

    # Display result
    plt.imshow(Image.open(file_path))
    plt.axis('off')
    plt.title(f"Prediction: {predicted_class} ({confidence:.2f}%)")
    plt.show()
else:
    print("No image selected.")
